## Compartmentalized FBA 
Making compartmentalized FBA models based on each of the 9 simulations.

In [ ]:
from cobra import Model
import pandas as pd
import numpy as np
import cobra as cb
import gifba
import optlang



for sit_idx in ["1c", "2a", "2b","2c", "2d","2e", "3a", "3b", "3c", "4a"]: 
	models, media = gifba.utils.load_simple_models(sit_idx)
	# rel abundances
	rel_abund = [0.5, 0.5] if sit_idx != "2b" else [0.2, 0.8]
	


	print(f"\nSimulation {sit_idx} Models:")
	for model_idx, model in enumerate(models):
		# these will be converted to the internal reactions between compartment e1 or e2 moving to e0
		for ex in model.exchanges:
			ex.lower_bound = 0  # Set lower bound to 0 for all exchange reactions
		for med_ex in media.keys():
			if med_ex in model.reactions:
				model.reactions.get_by_id(med_ex).lower_bound = media[med_ex] / rel_abund[model_idx]


	comp_model = Model(f"compartmentalized_sim{sit_idx}")
	met_mapping = dict({})
	#======= Change Compartments (each model uses e0 and c{i+1} compartments)============	
	for model_idx, model in enumerate(models):
		# Change compartments for each model so that models[i] uses e0 and c{i+1} compartments
		for met in model.metabolites:
			# store original compartment and id
			orig_comp = met.compartment
			orig_id = met.id

			# adjust new compartment and id
			if "bio" in met.id:
				id = met.id.replace(f"bio{model_idx +1}e", f"bio{model_idx +1}").replace(f"bio{model_idx +1}", f"bio{model_idx +1}_{orig_comp}{model_idx +1}")
			else:
				id = met.id.replace(f"_{orig_comp}", f"_{orig_comp}{model_idx +1}")
			met.id = id
			met.compartment = f"{orig_comp}{model_idx +1}"
			# map original id to new id
			met_mapping[id] = orig_id

			# add metabolite to compartmentalized model
			comp_model.add_metabolites([met.copy()])
	
	
	# change reaction names
	for model_idx, model in enumerate(models):
		for rxn in model.reactions:
			orig_id = rxn.id
			orig_mets = rxn.metabolites

			id = orig_id + f"_m{model_idx+1}"

			new_rxn = rxn.copy()
			new_rxn.id = id
			comp_model.add_reactions([new_rxn])
	
	for rxn in comp_model.reactions:
		if rxn.boundary:
			# copy original ex_met 
			met_og = list(rxn.metabolites.keys())[0]
			met_e0 = met_og.copy()

			# adjust to move met from e{i+1} to e0
			met_e0.compartment = "e0"
			met_e0.id = met_og.id.replace(f"_{met_og.compartment}", "_e0")
			model_num = int(met_og.compartment[-1]) -1
			comp_model.add_metabolites([met_e0])
			rxn.add_metabolites({
				met_e0: 1,
				# adjust for relative abundance and subtract 1 to account for original metabolite
				met_og.id: (-1/rel_abund[model_num] + 1) 
			})


	for met in comp_model.metabolites:
		if met.compartment == "e0":
			ex = cb.Reaction(f"EX_{met.id}")
			ex.name = f"Exchange for {met.id}"
			ex.lower_bound = -1000  # allow uptake
			ex.upper_bound = 1000   # allow secretion
			ex.add_metabolites({met: -1})
			comp_model.add_reactions([ex])

				


	
	for ex in comp_model.exchanges:
		ex.lower_bound = 0  # Set lower bound to 0 for all exchange reactions
		ex.upper_bound = 1000  # Set upper bound to 1000 for all exchange reactions

	for ex, flux in media.items():
		ex = ex.upper() + "_e0"
		comp_model.reactions.get_by_id(ex).lower_bound = flux  # set media uptake rates
		
	#========== Print compartmentalized model reactions =============
	for rxn in comp_model.reactions:
		# print original reaction
		print(rxn.id, ":  ", end="")
		for met in rxn.metabolites:
			if rxn.metabolites[met] < 0:
				print(rxn.metabolites[met], "*", met.id, end="  ")
		print(" --> ", end="")
		for met in rxn.metabolites:
			if rxn.metabolites[met] > 0:
				print(rxn.metabolites[met], "*", met.id, end="  ")

		print(" | LB:", rxn.lower_bound, " UB:", rxn.upper_bound)
		



	# Set objective to weighted sum of individual model biomass reactions
	objective_reactions = [comp_model.reactions.get_by_id(f"EX_bio{i+1}_e0") for i in range(len(models))]
	objective_rxns_coef = [1 for _ in range(len(models))]
	comp_model.objective = dict(zip(objective_reactions, objective_rxns_coef))



	# ======== Optimize compartmentalized model for (optimal) community growth ========
	solution = comp_model.optimize()
	cb.io.save_json_model(comp_model, f"Simple_Models/cFBA_Models/cFBA_sim{sit_idx}.json") # save for others to avoid re-building
	print(f"{comp_model.id}, Objective value: {solution.objective_value}")
	print(solution.fluxes)

	

	# ========= Set community growth constraint =============
	comm_growth = solution.fluxes[[rxn.id for rxn in objective_reactions]].sum()
	community_expr = sum(comp_model.reactions.get_by_id(f"EX_bio{i+1}_e0").flux_expression for i in range(len(models)))
	# Add constraint to model
	constraint = optlang.Constraint(
		community_expr,
		lb=comm_growth,  # lower bound
		ub=comm_growth,  # upper bound
		name="community_growth_constraint"
		)
	comp_model.solver.add(constraint)



	# ======= Get Min/Max Flux for organism 1 =============
	# minimize flux of first organism
	comp_model.objective = comp_model.reactions.get_by_id("EX_bio1_e0")
	comp_model.objective.direction = 'min'
	solution_min_s1 = comp_model.optimize()

	# maximize flux of first organism
	comp_model.objective.direction = 'max'
	solution_max_s1 = comp_model.optimize()

	# store results
	cfba_results = pd.DataFrame(columns=["First_Org_Flux", "Second_Org_Max_Flux", "Second_Org_Min_Flux"])

	

	# ======== Vary first organism flux between min and max and get second organism min/max ========
	for val in np.linspace(solution_min_s1.objective_value, solution_max_s1.objective_value, 5):
		# update constraint on first organism
		comp_model.reactions.get_by_id("EX_bio1_e0").upper_bound = val
		comp_model.reactions.get_by_id("EX_bio1_e0").lower_bound = val

		# optimize for second organism
		comp_model.objective = comp_model.reactions.get_by_id("EX_bio2_e0")
		comp_model.objective.direction = 'max'
		solution_max_s2 = comp_model.optimize()

		# minimize then maximize org 2
		comp_model.objective.direction = 'min'
		solution_min_s2 = comp_model.optimize()

		# store results
		cfba_results = pd.concat([cfba_results, pd.DataFrame({
			"First_Org_Flux": val,
			"Second_Org_Max_Flux": solution_max_s2.objective_value,
			"Second_Org_Min_Flux": solution_min_s2.objective_value,
		}, index=[0])], ignore_index=True)

	cfba_results.to_csv(f"Simple_Models/Results/cFBA/cFBA_results_sim{sit_idx}.csv", index=False)





Set parameter Username
Set parameter LicenseID to value 2684253
Academic license - for non-commercial use only - expires 2026-07-02

Simulation 1c Models:
Ex_A_m1 :  -2.0 * A_e1   --> 1 * A_e0   | LB: -20.0  UB: 1000.0
T_A_m1 :  -1 * A_e1   --> 1 * A_c1   | LB: -1000.0  UB: 1000.0
T_Bio1_m1 :  -1 * bio1_c1   --> 1 * bio1_e1   | LB: 0.0  UB: 1000.0
BiomassOrg1_m1 :  -1 * A_c1   --> 1 * bio1_c1   | LB: 0.0  UB: 1000.0
exchangeBio1_m1 :  -2.0 * bio1_e1   --> 1 * bio1_e0   | LB: 0  UB: 1000.0
Ex_B_m2 :  -2.0 * B_e2   --> 1 * B_e0   | LB: -20.0  UB: 1000.0
T_B_m2 :  -1 * B_e2   --> 1 * B_c2   | LB: -1000.0  UB: 1000.0
T_Bio2_m2 :  -1 * bio2_c2   --> 1 * bio2_e2   | LB: 0.0  UB: 1000.0
BiomassOrg2_m2 :  -1 * B_c2   --> 1 * bio2_c2   | LB: -1000.0  UB: 1000.0
exchangeBio2_m2 :  -2.0 * bio2_e2   --> 1 * bio2_e0   | LB: 0  UB: 1000.0
EX_A_e0 :  -1 * A_e0   -->  | LB: -10  UB: 1000
EX_bio1_e0 :  -1 * bio1_e0   -->  | LB: 0  UB: 1000
EX_B_e0 :  -1 * B_e0   -->  | LB: -10  UB: 1000
EX_bio2_e0 :  -

/tmp/ipykernel_54126/4050992455.py:179: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cfba_results = pd.concat([cfba_results, pd.DataFrame({
/tmp/ipykernel_54126/4050992455.py:179: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cfba_results = pd.concat([cfba_results, pd.DataFrame({
/tmp/ipykernel_54126/4050992455.py:179: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when d


Simulation 3b Models:
Ex_A_m1 :  -2.0 * A_e1   --> 1 * A_e0   | LB: -20.0  UB: 1000.0
Ex_B_m1 :  -2.0 * B_e1   --> 1 * B_e0   | LB: 0  UB: 1000.0
Ex_C_m1 :  -2.0 * C_e1   --> 1 * C_e0   | LB: 0  UB: 1000.0
Ex_D_m1 :  -2.0 * D_e1   --> 1 * D_e0   | LB: 0  UB: 1000.0
T_A_m1 :  -1 * A_e1   --> 1 * A_c1   | LB: -1000.0  UB: 1000.0
T_B_m1 :  -1 * B_c1   --> 1 * B_e1   | LB: -1000.0  UB: 1000.0
T_C_m1 :  -1 * C_e1   --> 1 * C_c1   | LB: -1000.0  UB: 1000.0
T_D_m1 :  -1 * D_c1   --> 1 * D_e1   | LB: -1000.0  UB: 1000.0
T_Bio1_m1 :  -1 * bio1_c1   --> 1 * bio1_e1   | LB: -1000.0  UB: 1000.0
BiomassFromA_m1 :  -1 * A_c1   --> 1 * B_c1  1 * bio1_c1   | LB: 0.0  UB: 1000.0
BiomassFromC_m1 :  -1 * C_c1   --> 1 * D_c1  1 * bio1_c1   | LB: 0.0  UB: 1000.0
exchangeBio1_m1 :  -2.0 * bio1_e1   --> 1 * bio1_e0   | LB: 0  UB: 1000.0
Ex_A_m2 :  -2.0 * A_e2   --> 1 * A_e0   | LB: -20.0  UB: 1000.0
Ex_B_m2 :  -2.0 * B_e2   --> 1 * B_e0   | LB: 0  UB: 1000.0
Ex_C_m2 :  -2.0 * C_e2   --> 1 * C_e0   | LB: 0  

/tmp/ipykernel_54126/4050992455.py:179: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cfba_results = pd.concat([cfba_results, pd.DataFrame({
/tmp/ipykernel_54126/4050992455.py:179: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cfba_results = pd.concat([cfba_results, pd.DataFrame({


## MICOM


In [ ]:
import pandas as pd
import gifba 
from micom import Community

micom_results = pd.DataFrame(columns=["simulation", "tradeoff", "Org1_growth", "Org2_growth"])

for sit_idx in ["1c","2a","2b", "2c", "2d", "2e", "3a", "3b", "3c"]: # "2a", "2b","3a", "3b", "3c"
	# get media from giFBA
	_ , media = gifba.utils.load_simple_models(sit_idx)
	media = {k.upper() + '_m': -v for k, v in media.items()}
	
	# make sure 2b (same model, different abundances) uses correct abundances
	abund = [0.2, 0.8] if sit_idx == "2b" else [0.5, 0.5]
	sim_idx = sit_idx if sit_idx != "2b" else "2a"

	# create community dataframe
	community = pd.DataFrame({
		"id": ["Org1", "Org2"],
		"file": [f"Simple_Models/Cobra_Models/sim{sim_idx}_1.json", f"Simple_Models/Cobra_Models/sim{sim_idx}_2.json"],
		"abundance": abund
	})

	# create micom community
	community = Community(community)
	community.medium = media
	community.solver = "gurobi"

	print(sit_idx)
	solutions = community.cooperative_tradeoff(fraction=[1, 0.8, 0.6, 0.4, 0.2, 0], pfba=True)
	for idx, sol in enumerate(solutions["solution"]):
		print(f"Solution alpha: {solutions['tradeoff'][idx]}", end="  ")
		# print(sol.members)
		org_growth = (sol.members["abundance"] * sol.members["growth_rate"]).to_numpy()[0:-1]
		print(org_growth)
		micom_results.loc[len(micom_results)] = [sit_idx, solutions['tradeoff'][idx], org_growth[0], org_growth[1]]

		
	print()

micom_results.to_csv("Simple_Models/Results/micom_results.csv", index=False)


Output()

[10/28/25 15:58:10] WARNING  There does not seem to be any carbon source in your medium. Please    ]8;id=827204;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py\community.py]8;;\:]8;id=186002;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py#667\667]8;;\
                             double-check your medium IDs.                                                         

Output()

1c
Solution alpha: 1.0  [10. 10.]
Solution alpha: 0.8  [8. 8.]
Solution alpha: 0.6  [6. 6.]
Solution alpha: 0.4  [4. 4.]
Solution alpha: 0.2  [2. 2.]
Solution alpha: 0.0  [0. 0.]



                    WARNING  There does not seem to be any carbon source in your medium. Please    ]8;id=682165;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py\community.py]8;;\:]8;id=721315;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py#667\667]8;;\
                             double-check your medium IDs.                                                         

...

                    WARNING  There does not seem to be any carbon source in your medium. Please    ]8;id=28122;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py\community.py]8;;\:]8;id=724024;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py#667\667]8;;\
                             double-check your medium IDs.                                                         

3b
Solution alpha: 1.0  [20. 20.]
Solution alpha: 0.8  [16.000015 15.999985]
Solution alpha: 0.6  [12.00001707 11.99998293]
Solution alpha: 0.4  [8.00002203 7.99997797]
Solution alpha: 0.2  [4.00003 3.99997]
Solution alpha: 0.0  [5.75027057e-06 3.45254828e-06]



Output()

                    WARNING  There does not seem to be any carbon source in your medium. Please    ]8;id=909575;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py\community.py]8;;\:]8;id=508060;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py#667\667]8;;\
                             double-check your medium IDs.                                                         

3c
Solution alpha: 1.0  [10. 10.]
Solution alpha: 0.8  [8. 8.]
Solution alpha: 0.6  [6. 6.]
Solution alpha: 0.4  [4. 4.]
Solution alpha: 0.2  [2. 2.]
Solution alpha: 0.0  [4.69546032e-06 5.13516786e-06]

